In [243]:
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import requests
import pandas as pd
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import datetime
import urllib.request
import dash_html_components as html
import dash_table
import dash_table.FormatTemplate as FormatTemplate
from dash_table.Format import Sign
from plotly.subplots import make_subplots
import ssl
import json
import time
import dash_trich_components as dtc
import random

In [298]:
px.colors.sequential.Plasma

['#0d0887',
 '#46039f',
 '#7201a8',
 '#9c179e',
 '#bd3786',
 '#d8576b',
 '#ed7953',
 '#fb9f3a',
 '#fdca26',
 '#f0f921']

>Dokumnetacja-> https://lunarcrush.com/developers/docs#


>login -> fozurek@wp.pl


> hasło -> StockApp

>widgets - > https://lunarcrush.com/developers/widgets

### Color asset

collors in chart = px.colors.sequential.Plasma_r

contrast white = #e5e9f0

dark background = #061e44

lighter background = #082255

light blue(same as table header) = #13326c


In [244]:
def get_coin_data(symbol, data_points=365, interval='day'):
    url = "https://api.lunarcrush.com/v2?data=assets&key=39o5gbbtx7n1gxkvnhf6na&symbol={}&data_points={}&interval={}".format(
        symbol, data_points, interval)
    response = requests.get(url).json()
    data = response['data'][0]
    df = pd.json_normalize(data['timeSeries'])
    df['time'] = pd.to_datetime(df['time'], unit='s')
    data = pd.json_normalize(data)
    df['MA-Low'] =df.low.rolling(window=20).mean()
    df['MA-High'] = df.high.rolling(window=20).mean()
    df['MA-Volume'] = df.volume.rolling(window=20).mean()
    details = data[
        ['name', 'symbol','price', 'price_btc', 'market_cap', 'percent_change_24h', 'percent_change_7d', 'percent_change_30d',
         'volume_24h', 'max_supply']]

    return (df, details.to_dict(orient='records')[0])


### Plots tab2 volume , indicator

In [245]:
df,details = get_coin_data('BTC')
df

,asset_id,time,open,close,high,low,volume,market_cap,url_shares,unique_url_shares,...,social_volume,price_btc,social_volume_global,social_dominance,market_cap_global,market_dominance,percent_change_24h,MA-Low,MA-High,MA-Volume
0,1,2020-05-20,9713.874440,9518.417317,9721.622118,9427.595340,40060697475,1.793943e+11,34727,21125,...,154276,1,279754,55.147022,264090932612,67.929007,-2.793307,NaN,NaN,NaN
1,1,2020-05-21,9363.038823,9074.884501,9382.962472,8924.955503,43383545146,1.718354e+11,33645,20506,...,158635,1,284908,55.679377,252484362702,68.057841,-4.659733,NaN,NaN,NaN
2,1,2020-05-22,9242.605652,9212.427524,9251.929883,9189.294287,34727139988,1.687968e+11,31757,19613,...,145993,1,268096,54.455494,258262913949,65.358511,1.515645,NaN,NaN,NaN
3,1,2020-05-23,9211.919234,9190.292929,9211.919234,9187.875905,30552307804,1.680265e+11,27511,16577,...,110321,1,225435,48.936944,257327863481,65.296671,-0.240269,NaN,NaN,NaN
4,1,2020-05-24,9057.232382,8744.687578,9061.691420,8744.687578,34171902172,1.685489e+11,31819,18185,...,123909,1,240140,51.598651,247179046498,68.188998,-4.848652,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,1,2021-05-15,49498.658905,47068.551165,50129.236422,46872.278849,67031619617,9.270091e+11,48797,24409,...,304985,1,1312057,23.244798,2316142289791,40.023843,-5.886289,54079.945162,56270.417536,7.283464e+10
361,1,2021-05-16,46935.379265,46663.010792,49300.559905,44511.848521,75387913732,8.874502e+11,53296,22939,...,485397,1,1591041,30.508139,2297927233726,38.619596,-0.861595,53800.859855,56037.687016,7.302892e+10
362,1,2021-05-17,46212.968046,43603.662573,46325.907795,42379.462085,89273851463,8.552181e+11,73646,35480,...,591822,1,1667658,35.488212,2150470336723,39.768888,-6.556260,53191.366324,55592.055307,7.456260e+10
363,1,2021-05-18,43719.099818,42945.731738,45481.150295,42642.280456,64432293900,8.291734e+11,62193,31243,...,407936,1,1587730,25.693033,2174455995960,38.132452,-1.508889,52582.192313,55076.833274,7.497415e+10


In [246]:
fig = px.bar(df[20:],
             x='time',
             y='volume',
             color='volume',
             color_continuous_scale=px.colors.sequential.Plasma,
             height=400,     
            )

fig2 = go.Figure(go.Scatter(x=df['time'],
                            y=df['MA-Volume'],
                            fill='tonexty',
                            mode='lines',
                            line=dict(width=0,
                                     color='rgb(0,212,255,1)',
                                     ),
                            opacity=0.01,
                            name='MV-Volume',
                            line_shape='spline'
                           ))
fig.update_yaxes(range=[0,200000000000])
fig.add_trace(fig2.data[0])
fig.update_layout(
        xaxis=dict(
            showline=False,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            title="Time",
            tickfont=dict(
                family='Arial',
                size=12,
                color='#e5e9f0',
            ),
        ),
        yaxis=dict(
            title="Volume",
            showgrid=True,
            zeroline=False,
            showline=False,
            showticklabels=True,
            gridcolor="#0f41a3",
            color='#e5e9f0'
        ),
        showlegend=True,
        plot_bgcolor='#082255',
        xaxis_rangeslider_visible=False,
        legend=dict(orientation="h",
                    yanchor="bottom",
                    y=1,
                    xanchor="right",
                    x=1,
                   ),
        height=300,
        coloraxis_showscale=False,
        
    )
fig.update_xaxes(fixedrange = True)
fig.update_yaxes(fixedrange = True)
fig.update_traces(
        marker=dict(
            line=dict(
            color='#0d378c')
        )
)

fig.update_layout(paper_bgcolor='#082255', font_color='#e5e9f0', width=1000)
fig.show()

### Indicator (tab2)

In [247]:
# fig3 = go.Figure(go.Indicator(
#     mode = "number+delta",
#     value= details['price'],
#     title= "{} 24 hours".format(details['name']),
#     delta = {'reference':(1 + (-1* (details['percent_change_24h']/100))) * details['price'], 'relative': True},
    
# ))
# fig3.show()

### Data and chart to 1 tab 

In [248]:
def get_global_information(change=6,data_points=182):
    url = "https://api.lunarcrush.com/v2?data=global&key=39o5gbbtx7n1gxkvnhf6na&change={}m&data_points={}".format(change,data_points)
    response = requests.get(url).json()
    df = pd.json_normalize(response['data']['timeSeries'])
    df = df[['time','volume','btc_dominance','btc_market_cap','alt_coin_market_cap','altcoin_dominance']]
    df['time'] = pd.to_datetime(df['time'], unit='s') 
    return df

In [249]:
df = get_global_information(change=6,data_points=182)


In [296]:
def meanwhile():
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df['time'],
                            y=df['btc_dominance'],
                            name= 'BTC dominance',
                            line=dict(color='#d8576b',
                                     width=3,
                                     )

                            ),secondary_y = False)
    fig.add_trace(go.Scatter(x=df['time'],
                            y=df['altcoin_dominance'],
                            name= 'Altcoin dominance',
                            line=dict(color='#bd3786',
                                      width=3,
                                     )
                            ),secondary_y = False)
    fig.add_trace(go.Scatter(x=df['time'],
                            y=df['alt_coin_market_cap'],
                            name= 'Altcoin market cap',
                             line=dict(color= '#fdca26',
                                      width=3,)
                            ),secondary_y = True)
    fig.add_trace(go.Scatter(x=df['time'],
                            y=df['btc_market_cap'],
                            name= 'BTC market cap',
                             line=dict(color='#f0f921',
                                      width=3,)
                            ),secondary_y = True)

    fig.update_yaxes(title_text="<b>Dominance</b>", secondary_y=False,showline=False,
                showgrid=False,)
    fig.update_yaxes(title_text="<b>Market cap</b>", secondary_y=True,showline=False,
                showgrid=False,)
    fig.update_layout(
            xaxis=dict(
                showline=False,
                showgrid=False,
                showticklabels=True,
                linecolor='rgb(204, 204, 204)',
                linewidth=2,
                ticks='outside',
                title="Time",
                tickfont=dict(
                    family='Arial',
                    size=12,
                    color='#e5e9f0',
                ),
            ),
            yaxis=dict(
                showgrid=False,
                zeroline=False,
                showline=False,
                showticklabels=True,
                gridcolor="#0f41a3",
                color='#e5e9f0'
            ),
        showlegend=True,
        plot_bgcolor='#082255'
    )
    fig.update_layout(legend=dict(orientation="h",
                    yanchor="bottom",
                    y=1,
                    xanchor="right",
                    x=1,
                                  font=dict(size=10,)
                   ),
)
    fig.update_xaxes(fixedrange = True)
    fig.update_layout(paper_bgcolor='#082255', font_color='#e5e9f0', width=700, height = 450)
    return fig

In [297]:
meanwhile()

In [252]:
def get_n_top_coins(n=50):
    url = 'https://api.lunarcrush.com/v2?data=market&key=39o5gbbtx7n1gxkvnhf6na&limit={}&sort=acr&type=fast'.format(n)
    response = requests.get(url).json()
    df = pd.json_normalize(response['data'])
    df.rename(columns={
        's': 'Symbol',
        'n': 'Name',
        'p': 'Price',
        'v': 'Volume',
        'pc': '24 Hours',
        'pch': '1 Hour',
        'mc': 'Market Cap',
        'gs': 'Galaxy Score',
        'ss': 'Social Score',
        'acr': 'ALTRank',
        't': 'Number of tweets',
        'sd': 'Social Dominance',
        'd': 'Market Dominance'
    }, inplace=True)
    df['24 Hours'] = df['24 Hours'].div(100).round(4)
    df['1 Hour'] = df['1 Hour'].div(100).round(4)

    return df[['ALTRank', 'Galaxy Score', 'Social Score', 'Symbol', 'Name', 'Price', 'Volume', '24 Hours', '1 Hour',
               'Market Cap', 'Number of tweets', 'Social Dominance', 'Market Dominance']]


In [253]:
def get_market_table(n=50):
    market_data = get_n_top_coins(n)
    market_table = dash_table.DataTable(
        id='market_table',
        data=market_data.to_dict('records'),
        columns=[
            {'id': 'ALTRank', 'name': 'ALTRank', 'type': 'numeric'},
            {'id': 'Galaxy Score', 'name': 'Galaxy Score (PT)',
             'type': 'numeric'},
            {'id': 'Social Score', 'name': 'Social Score (PT)',
             'type': 'numeric'},
            {'id': 'Symbol', 'name': 'Symbol', 'type': 'text'},
            {'id': 'Name', 'name': 'Name', 'type': 'text'},
            {
                'id': 'Price',
                'name': 'Price',
                'type': 'numeric',
                'format': FormatTemplate.money(6),
            },
            {
                'id': 'Volume',
                'name': 'Volume',
                'type': 'numeric',
                'format': FormatTemplate.money(2),
            },
            {
                'id': '24 Hours',
                'name': '24 Hours',
                'type': 'numeric',
                'format': FormatTemplate.percentage(2).sign(Sign.positive),
            },
            {
                'id': '1 Hour',
                'name': '1 Hour',
                'type': 'numeric',
                'format': FormatTemplate.percentage(2).sign(Sign.positive),
            },
            {
                'id': 'Market Cap',
                'name': 'Market Cap',
                'type': 'numeric',
                'format': FormatTemplate.money(0),
            },
        ],
        style_data_conditional=[
            #         data_bars(market_data, 'Volume')[0],
            {'if': {'filter_query': '{24 Hours} > 0',
                    'column_id': '24 Hours'}, 'color': 'green',
             'fontWeight': 'bold'},
            {'if': {'filter_query': '{24 Hours} < 0',
                    'column_id': '24 Hours'}, 'color': 'red', 'fontWeight': 'bold'
             },
            {'if': {'filter_query': '{24 Hours} = 0',
                    'column_id': '24 Hours'}, 'color': 'blue', 'fontWeight': 'bold'
             },
            {'if': {'filter_query': '{1 Hour} > 0', 'column_id': '1 Hour'},
             'color': 'green', 'fontWeight': 'bold'},
            {'if': {'filter_query': '{1 Hour} < 0', 'column_id': '1 Hour'},
             'color': 'red', 'fontWeight': 'bold'},
            {'if': {'filter_query': '{1 Hour} = 0', 'column_id': '1 Hour'},
             'color': 'blue', 'fontWeight': 'bold'},
            {'if': {'row_index': 'odd'},
             'backgroundColor': '#061e44'},
        ],
        style_cell_conditional=[{'if': {'column_id': str('Name')},
                                 'textAlign': 'left'},
                                #                             {
                                #         'if': {'column_id': 'Volume'},
                                #         'width': '200px',
                                #         'color': 'blue',
                                #         'minWidth': '200px',
                                #         'maxWidth': '200px',
                                #         'overflow': 'hidden',
                                #         'textOverflow': 'ellipsis',},
                # nie mam pojecia czemu to nie dziala ale trzeba to zmienic!!!
#                                 {'if': {'column_id': (str('Symbol'), str('ALTRank'), str('Galaxy Score'))},
#                                  'fontWeight': 'bold',
#                                  'textAlign': 'center'}
                               ],
        style_header={'backgroundColor': '#13326c',
                      'fontWeight': 'bold',
                      'fontSize': 17,
                      },
        style_cell={'fontSize': 15,
                    'font-family': 'Lucida Console',
                    'backgroundColor': "#082255",
                    'color': '#e5e9f0',
                    'border': '1px solid #082255'},
        # jakiś error podczas sortowania np 24 hours lub 1houur
        sort_action='native',
        editable=False,
        style_as_list_view=True,
        page_size=5,
        #     fill_width=False
    )
    return market_table

In [254]:
def build_header():
    return html.Div([
        html.Div([
            html.H4("Our Title", className='app-header-title'),
            html.P("Some descriptons, maybe our names or somethings like this", 'app-header-subtitle'),
        ], className='app-header-main'),
        html.Img(
            src='assets/crunchips.png',
            className='logo'
        ),
        html.Audio(id='easter-egg', controls=True, autoPlay=True, hidden=True)
    ], className='app-header')

In [255]:
def build_tabs():
    return html.Div(
        id='tabs',
        className="tabs",
        children=[
            dcc.Tabs(
                id='control-tabs',
                value='tab1',
                className="custom-tabs",
                children=[
                    dcc.Tab(
                        id='Main-tab',
                        label='Main Tab',
                        value="tab1",
                        className="custom-tabs",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id='Sub-tab',
                        label='Sub Tab',
                        value="tab2",
                        className="custom-tabs",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id='Rest-tab',
                        label='Rest Tab',
                        value="tab3",
                        className="custom-tabs",
                        selected_className="custom-tab--selected",
                    ),
                ],
            )
        ],
    )

In [256]:
def get_social_coin_data(symbol):
    url = 'https://api.lunarcrush.com/v2?data=assets&key=39o5gbbtx7n1gxkvnhf6na&symbol={}'.format(symbol)
    response = requests.get(url).json()['data']

    df = pd.json_normalize(
        response,
        record_path=['timeSeries'],
        meta=['symbol']
    )
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df[['unique_url_shares', 'url_shares', 'reddit_posts', 'tweets', 'tweet_spam', 'time', 'symbol']]
    return df

In [257]:
def generate_type_dropdown():
    return dcc.Dropdown(id='types',
                 className='dropdown-type',
                 value='Market Dominance',
                 options=[{'label': x,
                           'value': x,
                           } for x in ['Market Dominance',
                                       'Social Dominance',
                                       'Number of tweets']],
                 clearable=False,
                 # trzeba pozmieniac jakos na naturalne kolory te dropdowny 
                 style={
                     'font-family': 'Lucida Console',
                     'color': '#212121',
                     'border': '0px',

                 })
    


In [258]:
def generate_symbol_dropdown():
    return dcc.Dropdown(id='selected_coins',
             className='dropdown-top',
             value=coins[:3],
             options=options,
             multi=True,
             style={'color': '#212121',
                    })

In [259]:
def generate_coin_dropdown():
    return dcc.Dropdown(
            id='coin-dropdown',
            clearable=False,
            value='BTC',
            options=[{'label': c, 'value': c} for c in coins],
            style={'color': '#212121'})

In [270]:
def build_tab_1():
    return [
        dcc.Loading(id='loading_tab1',
                       children=[
        html.Div([
            html.Div([
                get_market_table()], 
                className='tabble-wrapper' ),
            
            html.Div([
                html.Div([
                    html.Div([
                        html.P("Type:"),
                        generate_type_dropdown(),
                        html.P("Enter a stock symbol:"),
                        generate_symbol_dropdown(),
                            ], className='dropdowns-wrapper'),
                    html.Div([
                        dcc.Graph(id="pie-chart")],
                        className='gauge-wrapper'),
                    
                ], className='leftcolumn-wrapper'),
                html.Div([
                    html.Div([
                        dcc.Graph(id="scatter-chart")],
                        className='scatter-wrapper'),
                    html.Div([
                        dcc.Graph(id='dominance-line')
                    ], className = 'dominance-wrapper')
                    
                ], className='rightcolumn-wrapper'),
            ], className='bottom-wrapper')

        ], className='first-div')],type='cube')
    ]

In [271]:
def build_tab_2():
    return [
        dcc.Loading(id='loading_tab1',
                       children=[
        html.Div([ #second div
            html.Div([ #left column
                html.Div([#topleft wrapper
                    html.Div([
                        html.Label("Coin"),
                        generate_coin_dropdown()
                    ], className='dropdowns2-wrapper'),
                    html.Div([
                        dcc.Graph(id="indicator")
                    ],className='indicator')
                    
                ], className = 'topleft-wrapper'),
                html.Div([
                    html.Div([
                        dcc.Graph(id='graph-social')
                    ], className='plotline-wrapper'), 
                    
                ], className = 'bottomleft-wrapper')
            ], className='leftcolumn2-wrapper'),
            
            html.Div([
                html.Div([
                    dcc.Graph(id='graph')
                ], className='plotline3-wrapper'),
                html.Div([
                    dcc.Graph(id='volumeplot')
                ], className='volumeplot-wrapper')
            ], className='rightcolumn2-wrapper')    
            
        ], className='second-div')],type='cube',style=dict(color="red"))
    ]

In [272]:
def get_feeds_information():
    url = "https://api.lunarcrush.com/v2?data=feeds&key=39o5gbbtx7n1gxkvnhf6na&limit=20&sources=news,urls"
    response = requests.get(url).json()
    df = pd.json_normalize(response['data'])
    df['time'] = pd.to_datetime(df['time'], unit='s') 
    return df

In [273]:
def build_card(links, image_path, titles, desc, source, who, symbol):
    return dtc.Card(
        link=links,
        image=image_path,
        title=titles,
        description=desc,
        badges=[source, who, symbol],
        dark=True 
    )

In [274]:
feeds = get_feeds_information()
lists = [i for i in range(20)]
random.shuffle(lists)

In [275]:
def create_card(ids):
    dfo = feeds.iloc[lists[ids]]
    return build_card(dfo['url'], dfo['image'], dfo['title'], dfo['description'], 
                      dfo['type'], dfo['publisher'], dfo['symbol'])

In [276]:
def build_tab_3():
    return [
        dcc.Loading(children=[
        html.Div([ # third-div
            html.Div([#card-container
                html.Div([#card-box
                    create_card(0),
                ], className = 'card-box'),
                html.Div([#card-box
                    create_card(1),
                ], className = 'card-box'),
                html.Div([#card-box
                    create_card(2),
                ], className = 'card-box'),
                html.Div([#card-box
                    create_card(3),
                ], className = 'card-box'),
                
            ],className = 'card-container')

        ], className = 'third-div')],type='cube')
    ]

In [277]:
z = get_n_top_coins(n=50)
coins = z['Symbol'].to_list()
options = []
for coin in coins:
    options.append({'label': coin, 'value': coin})



In [278]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        build_header(),
        build_tabs(),
        html.Div(id='app-content')
    ], style={'padding': 20})

])


@app.callback(
    Output('app-content', 'children'),
    Input('control-tabs', 'value')
)
def prepare_top(tab_switch):
    if tab_switch == 'tab1':
        return build_tab_1()
    elif tab_switch == 'tab2':
        return build_tab_2()
    else:
        return build_tab_3()


@app.callback(
    [Output("pie-chart", "figure"),
     Output("scatter-chart", "figure")],
    [Input("types", "value"),
     Input('selected_coins', 'value')])
def generate_chart(types, selected_coins):
    df = get_n_top_coins(50)
    df = df[df['Symbol'].isin(selected_coins)]
    fig_pie = px.pie(df,
                     values=df[types],
                     names=df['Symbol'],
                     hole=+.5,
                     title=types,
                     color_discrete_sequence=px.colors.sequential.Plasma_r)

    fig_pie.update_layout(plot_bgcolor='#082255', paper_bgcolor='#082255', font_color='#FFFFFF', 
                          width=300, height =250, margin_b = 0, margin_t = 20)

    fig_scatter = px.scatter(df,
                             x=df['Price'],
                             y=df['Volume'],
                             size=df[types],
                             color=df['Symbol'],
                             hover_name=df['Symbol'],
                             log_y=True,
                             log_x=True,
                             size_max=100,
                             color_discrete_sequence=px.colors.sequential.Plasma_r)
    fig_scatter.update_layout(title=types,
                              xaxis_title="Price ($)",
                              yaxis_title="Volume($)",
                              plot_bgcolor='#082255',
                              )

    fig_scatter.update_layout(paper_bgcolor='#082255',
                              font_color='#FFFFFF',
                              xaxis=dict(showgrid=False),
                              yaxis=dict(gridcolor="#0f41a3"),
                              width = 500,
                              height = 450
                              )

    return fig_pie, fig_scatter


@app.callback(
    Output('easter-egg', 'src'),
    Input('coin-dropdown', 'value')
)
def easteregg(coin):
    if coin =='DOGE':
        return 'assets/dog.wav'
    else:
        return ''
    
@app.callback(
    [Output('graph', 'figure'),
     Output('graph-social', 'figure'),
     Output('indicator', 'figure')],
    [Input('coin-dropdown', 'value')]
)
def update_figure(coin):
    x, details = get_coin_data(coin)
    fig = go.Figure(data=[go.Candlestick(x=x['time'],
                                         open=x['open'],
                                         high=x['high'],
                                         low=x['low'],
                                         close=x['close'],
                                         name='{} price'.format(details['symbol']))])

    fig.update_layout(title=details['name'],
                      yaxis_tickformat='$',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      )

    fig.update_xaxes(title_text='Date',
                     rangeslider_visible=True,
                     rangeselector=dict(buttons=list([dict(count=1,
                                                           label='1M',
                                                           step='month',
                                                           stepmode='backward'),
                                                      dict(count=6,
                                                           label='6M',
                                                           step='month',
                                                           stepmode='backward'),
                                                      dict(count=1,
                                                           label='YTD',
                                                           step='year',
                                                           stepmode='todate'),
                                                      dict(count=1,
                                                           label='1Y',
                                                           step='year',
                                                           stepmode='backward'),
                                                      dict(step='all')]), bgcolor=' #061e44'))
    fig.add_trace(go.Scatter(x=x['time'],
                             y=x['MA-Low'],
                             mode='lines',
                             name='Low-Moving Average',
                             line={'color': '#ff9100',
                                   'width': 1}))
    fig.add_trace(go.Scatter(x=x['time'],
                             y=x['MA-High'],
                             mode='lines',
                             name='High-Moving Average',
                             line={'color': '#006eff',
                                   'width': 1}))
    fig.update_layout(
        xaxis=dict(
            showline=False,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=12,
                color='#e5e9f0',
            ),
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=False,
            showline=False,
            showticklabels=True,
            gridcolor="#0f41a3",
            color='#e5e9f0'
        ),
        showlegend=True,
        plot_bgcolor='#082255',
        xaxis_rangeslider_visible=False
    )
    fig.update_layout(paper_bgcolor='#082255', font_color='#FFFFFF', width=1000)

    
    
    df = get_social_coin_data(coin)
    colors = px.colors.sequential.Plasma_r
    fig1 = go.Figure()
    labels = ['unique_url_shares', 'url_shares', 'tweets', 'tweet_spam']
    for j, i in enumerate(labels):
        fig1.add_trace(go.Scatter(x=df['time'], y=df[i], mode='lines',
                                  name=labels[j],
                                  line=dict(color=colors[j], width=4),
                                  connectgaps=True,

                                  ))

    fig1.update_layout(
        xaxis=dict(
            showline=False,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Lucida Console',
                size=12,
                color='#e5e9f0',
            ),
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=False,
            showline=False,
            showticklabels=True,
            color='#e5e9f0',
            gridcolor="#0f41a3",
        ),
        autosize=False,
        title="24 hours {} social".format(coin),
        xaxis_title="Date",
        yaxis_title="Number",
        showlegend=True,
        plot_bgcolor='#082255',
        paper_bgcolor='#082255',
        font_color='#e5e9f0'
    )
    
    fig1.update_layout(legend=dict(
        yanchor="top",
        y=1.5,
        xanchor="right",
        x=1.1
    ))
    
    fig3 = go.Figure(go.Indicator(
        mode = "number+delta",
        value= details['price'],
        title= "{} 24 hours".format(details['name']),
        delta = {'reference':((1 + (-1* (details['percent_change_24h']/100))) * details['price']), 'relative': True},
        
        
    
    ))
    fig3.update_layout(
        plot_bgcolor='#082255',
        paper_bgcolor='#082255',
        font_color='#e5e9f0',
        width=250,
        height=250,
        
            )

    return fig, fig1,fig3

@app.callback(
    Output("dominance-line", "figure"),
    [Input("types", "value"),
     Input('selected_coins', 'value')]
)
def meanwhile(types, coin):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df['time'],
                            y=df['btc_dominance'],
                            name= 'BTC dominance',
                            line=dict(color='#d8576b',
                                     width=3,
                                     )

                            ),secondary_y = False)
    fig.add_trace(go.Scatter(x=df['time'],
                            y=df['altcoin_dominance'],
                            name= 'Altcoin dominance',
                            line=dict(color='#bd3786',
                                      width=3,
                                     )
                            ),secondary_y = False)
    fig.add_trace(go.Scatter(x=df['time'],
                            y=df['alt_coin_market_cap'],
                            name= 'Altcoin market cap',
                             line=dict(color= '#fdca26',
                                      width=3,)
                            ),secondary_y = True)
    fig.add_trace(go.Scatter(x=df['time'],
                            y=df['btc_market_cap'],
                            name= 'BTC market cap',
                             line=dict(color='#f0f921',
                                      width=3,)
                            ),secondary_y = True)

    fig.update_yaxes(title_text="<b>Dominance</b> yaxis title", secondary_y=False,showline=False,
                showgrid=False,)
    fig.update_yaxes(title_text="<b>Market cap</b> yaxis title", secondary_y=True,showline=False,
                showgrid=False,)
    fig.update_layout(
            xaxis=dict(
                showline=False,
                showgrid=False,
                showticklabels=True,
                linecolor='rgb(204, 204, 204)',
                linewidth=2,
                ticks='outside',
                title="Time",
                tickfont=dict(
                    family='Arial',
                    size=12,
                    color='#e5e9f0',
                ),
            ),
            yaxis=dict(
                showgrid=False,
                zeroline=False,
                showline=False,
                showticklabels=True,
                gridcolor="#0f41a3",
                color='#e5e9f0'
            ),
        showlegend=True,
        plot_bgcolor='#082255'
    )
    fig.update_layout(legend=dict(
    yanchor="top",
    y=1.3,
    xanchor="right",
    x=1.1
))
    fig.update_xaxes(fixedrange = True)
    fig.update_layout(paper_bgcolor='#082255', font_color='#e5e9f0', width=700, height = 450)
    return fig

@app.callback(
    [Output('volumeplot', 'figure')],
    [Input('coin-dropdown', 'value')]
)
def somename(coin):
    df,details = get_coin_data('BTC')
    fig = px.bar(df[20:],
                 x='time',
                 y='volume',
                 color='volume',
                 color_continuous_scale=px.colors.sequential.Plasma,
                 height=400,     
                )

    fig2 = go.Figure(go.Scatter(x=df['time'],
                                y=df['MA-Volume'],
                                fill='tonexty',
                                mode='lines',
                                line=dict(width=0,
                                         color='rgb(0,212,255,1)',
                                         ),
                                opacity=0.01,
                                name='MV-Volume',
                                line_shape='spline'
                               ))
    fig.update_yaxes(range=[0,200000000000])
    fig.add_trace(fig2.data[0])
    fig.update_layout(
            xaxis=dict(
                showline=False,
                showgrid=False,
                showticklabels=True,
                linecolor='rgb(204, 204, 204)',
                linewidth=2,
                ticks='outside',
                title="Time",
                tickfont=dict(
                    family='Arial',
                    size=12,
                    color='#e5e9f0',
                ),
            ),
            yaxis=dict(
                title="Volume",
                showgrid=True,
                zeroline=False,
                showline=False,
                showticklabels=True,
                gridcolor="#0f41a3",
                color='#e5e9f0'
            ),
            showlegend=True,
            plot_bgcolor='#082255',
            xaxis_rangeslider_visible=False,
            legend=dict(orientation="h",
                        yanchor="bottom",
                        y=1,
                        xanchor="right",
                        x=1,
                       ),
            height=300,
            coloraxis_showscale=False,

        )
    fig.update_xaxes(fixedrange = True)
    fig.update_yaxes(fixedrange = True)
    fig.update_traces(
            marker=dict(
                line=dict(
                color='#0d378c')
            )
    )

    fig.update_layout(paper_bgcolor='#082255', font_color='#e5e9f0', width=1000)
    return fig


app.run_server(mode='external',debug=True )

Dash app running on http://127.0.0.1:8050/
